In [10]:
import numpy as np 
import pandas as pd 

### LOAD DATA

In [ ]:
import pandas as pd 

def data_load(path_to_rating, nb_reviews):
    with open(path_to_rating, encoding="utf-8") as input_file:
        rb_ratings = [next(input_file) for _ in range(nb_reviews)]

    rb_ratings_text = [x.replace("text: ", "")
                  .replace("\n", "")
                  for x in rb_ratings 
                  if x.startswith("text:")]

    rb_ratings_num = [round(float(x.replace("rating: ", "")
                    .replace("\n", "")))
                    for x in rb_ratings 
                    if x.startswith("rating:")]

    rb_beer_name = [x.replace("beer_name: ", "")
                    .replace("\n", "")
                    for x in rb_ratings 
                    if x.startswith("beer_name:")] 
    
    data = {'review': rb_ratings_text, 'score': rb_ratings_num, "beer_name" : rb_beer_name} 

    return pd.DataFrame(data)

In [51]:
nb_reviews = 5882 * 17 # about 100K lines (17 is the number of lines for one review for RateBeer)
path_to_rating = "data/RateBeer/ratings.txt"
data = data_load(path_to_rating, nb_reviews)

### REMOVE NON ENGLISH REVIEWS

In [52]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii') # detect all non ascii characters => most of the time not english
    except UnicodeDecodeError:
        return False
    else:
        return True

non_english_lines = [ind for ind, rev in enumerate(data["review"]) if isEnglish(rev)]

In [53]:
data = data.loc[non_english_lines]
len(data)

4496

### EXTRACTING INFO ABOUT BEER FROM REVIEW (TASK 1)

### EMOTIONS REVIEW

In [ ]:
from transformers import pipeline

def emotion_sentiment(data, cuda=False):
    if cuda:
        classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None, decice="cuda")
    else:
        classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)
        
    emotions = {"anger" : [], "disgust": [], 'fear' : [], 'joy' : [], 'neutral' : [], 'sadness' : [], 'surprise' : []}
    total = len(data)

    for x in data["review"]:
        if len(x) < 500: # The length constraint is because the model has a length constraint
            result = classifier(x)[0]
            for emotion in result:
                emotions[emotion["label"]].append(emotion["score"])
            if len(emotions["anger"]) % 100 == 0:
                print(f"{(len(emotions['anger'])/total * 100):.2f} % done", end='\r') 
        else:
            for emotion in result:
                emotions[emotion["label"]].append(0)

    for label, values in emotions.items():
        data[label] = values

    # Create a additionnal column representing the feeling with the maximum score according to the model
    data["max_feel"] = data[["anger", "disgust", 'fear', 'joy', 'neutral', 'sadness', 'surprise']].idxmax(axis=1)
    
    return data

In [ ]:
data = emotion_sentiment(data)

Spearman correlation for emotions by score: 
(if p value < 0.05 we can say that a change in score, introducted a predictable increase or decrease in the percentage for that emotion) 
          Spearman correlation   p-value
anger                 0.542857  0.265703
disgust              -1.000000  0.000000
fear                 -0.714286  0.110787
joy                   1.000000  0.000000
neutral               0.942857  0.004805
sadness              -0.289886  0.577352
surprise             -0.115954  0.826848


In [40]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Compute counts for each score and emotion
emotion_counts = data.groupby(['score'])["max_feel"].value_counts().unstack(fill_value=0)
# Then their percentage in each score class
emotion_percentages = emotion_counts.div(emotion_counts.sum(axis=1), axis=0) * 100

emotion_percentages.plot(kind='bar', stacked=True, colormap="Set2", figsize=(10, 6))
plt.title("Percentage of found emotions in each score class")
plt.xlabel("Score")
plt.ylabel("Emotion percentage")
plt.legend(title="Emotion", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.xticks(rotation=0)
plt.show()

emotion_percentages = emotion_percentages.reset_index()

# Compute the spearman correlation for each emotion between its percentage and its score
spearman_results = {}
for emotion in emotion_counts.columns:
    spearman_corr, p_value = stats.spearmanr(emotion_percentages['score'], emotion_percentages[emotion])
    spearman_results[emotion] = {'Spearman correlation': spearman_corr, 'p-value': p_value}

spearman_df = pd.DataFrame(spearman_results).T
print("Spearman correlation for emotions by score: ")
print("(if p value < 0.05 we can say that a change in score, introducted a predictable increase or decrease in the percentage for that emotion) ")
print(spearman_df)


We can therefore use this model to get negative reviews even though the review are positive by filtering on emotions as depicted below where we are seaching for reviews that are classified as sad. We observe that the review are of course positive but still have some complaint or advice for improvement.

In [48]:
data[((data["score"] == 5) | (data["score"] == 4)) & (data["max_feel"] == "sadness")]["review"].iloc[:10].tolist()

['A bit mnah mnah in aftertaste and tchup tchup roast but the khhhhhh thickness feels fnnnnnnn ',
 'from cellar at standing order ... deep red brown... thin lacing ... light meaty ... big chocolate nose ... still little green ... but good solid roast ... little chocolate ... little woody ... light hop in finish ... tk 2 dark red brown ... creamy tan head ... soft roast malts ... thin roast ... light sourness... roast malt ',
 'Bottle shared at green sheep. Hazy golden colour. Nose is citruss and zest. Taste is also citrussy, zesty, with a nice bitter finish. Sadly the aroma faded quickly. Then it was just a medium bitter ipa. ',
 'Cask, The Crown, Belfast.  Cloudy somewhat gloomy appearance, think it was brown, but hard to tell under those gas lamps...Some nice fragrant hops in the aroma, with caramel malt and nuts too. Similar themes resonated in the flavour which was surprisingly complex.  Dryish,  gently bitter finish.  Really quite good. ',
 '@ A day to sink all seasheppards. Thnx 

### Defining Negative reviews (TAKS 2)

##### NEGATIVITY SCORE

In [54]:
from transformers import pipeline

def polarity_sentiment(data, cuda=False):
    # We consider negative review to be those with scores 0, 1 or 2, and we will try to find those that have an extreme vocabulary using a 
    # sentiment analysis model
    if cuda:     
        sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device="cuda")
    else:    
        sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

    negative_reviews = data[(data["score"] == 0) | (data["score"] == 1) | (data["score"] == 2)].copy()

    total = len(negative_reviews)
    polarity = []
    for x in negative_reviews["review"]:
        if len(x) < 500:
            result = sentiment_task(x)[0]
            if result["label"] != "negative":
                polarity.append(0)
            else:
                polarity.append(result["score"])
            if len(polarity) % 100 == 0:
                print(f"{(len(polarity)/total * 100):.2f} % done", end='\r') 
        else:
            polarity.append(0)
        
    negative_reviews["polarity"] = polarity

    return negative_reviews

In [55]:
negative_reviews = polarity_sentiment(data)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [56]:
top_negative_rev = (-negative_reviews[negative_reviews["polarity"] != 0.0]["polarity"]).argsort()

In [57]:
# 10 most negative reviews according to our model
negative_reviews[negative_reviews["polarity"] != 0.0]["review"].iloc[top_negative_rev[:10]].tolist()

['This is a terrible beer and should be avoided at all costs! The ginseng is so overpowering and gross that it totally kills this beer. Some things just should not be infused into beer! ',
 'oh wow, this is awful. Aroma and flavour smacks of adjuncts. Crappy vegetal, grainy flavour, blech, bad stuff.  ',
 'not very good at all. looks nice, but tastes awful. golden in colour, big white head, horrible taste.  ',
 'Can from FairPrice in Singapore. Golden pour with a huge white head. Plastic, rotten veggies, a drier finish - this is a special kind of bad. ',
 'Meh, very bland lager, luckily the bottle cost be just 69p! Just about drinkable but nothing very enjoyable about it at all. Grainy, grassy, bitter and blande. Very pale in colour too. ',
 'found this in vietnam.  holy bad beer.  this one pours just like a weak looking pale lager then you drink it and it overfows with bad off flavors and overpowering alcohol presence.   cereal backbone of graininess, slighlty sweet. ',
 '500 ml bottl

In [58]:
# 10 most "positive" reviews (among negatives) according to our model
negative_reviews[negative_reviews["polarity"] != 0.0]["review"].iloc[top_negative_rev[-10:]].tolist()

['thin feel, rather punget smell standard lager smell mixed with a strong smell of sour malt, Terrable flavor sweetness of a lager mixed with a stronger than usual alcohol taste (odd for such a low ABV) stadard hohum lager color. Stick to a nice pils .   ',
 'Amber-ish color with a fine off white head. Dried fruits and to much yeast in this one. Medium + body and a soft carbonation. Tad sweet and a moderate bitterness. Nah not god at all.[On tap at Brewbot in Belfast, Northern Ireland] ',
 'A skunky lager with a touch of some citrusy malt which seemed to go well with Chinese food but unfortunately never seemed to go away afterwards. ',
 "Not much to say...it's beer, but definitely nothing special. If I was living in Singapore and this was the only beer around, I'd love it. Since I don't, it's not exactly my top pick. ",
 'pale, light golden colour.  A sweet fruitiness to it, like a british ale gone bad.  Too fruity, needs more hops.  ',
 'No Head, No Aroma. Really just an innoffensive 

We observe a change in vocabulary used between the two lists of reviews, indicating the model is capable of finding reviews that use a more negative vocabulary.

##### 10 % across beers

In [ ]:
# Here are the 10 % worst reviews 

thresholds = data.groupby("beer_name")["score"].quantile(0.1)

# We get the 10 % lowest score review by adding the threshold column to each beer and comparing the scores for each review to the treshold
low_score_reviews = data.join(thresholds, on="beer_name", rsuffix="_threshold")
low_score_reviews = low_score_reviews[low_score_reviews["score"] <= low_score_reviews["score_threshold"]]

low_score_reviews = low_score_reviews.drop(columns=["score_threshold"])

print(low_score_reviews)

                                                 review  score  \
4     Many thanks for this beer to Erzengel. Pours l...      2   
5     Many thanks to Erzengel for sharing! Pours cle...      2   
8     330ml bottle. 29-05-2014.From Gabon, courtesy ...      2   
9     Light yellow with quickly disappearing foam to...      2   
10    Pours yellow with a large white head.  Smells ...      2   
...                                                 ...    ...   
5828  We have a theory here that the reason why they...      0   
5835  This beer finish is dry, medium fullness and s...      1   
5842  Basically water. Almost no aroma. Bitterness v...      1   
5865  Super-clear and golden-straw in color. Has a q...      1   
5875        Bland with a sharp, unpleasant bitterness.       1   

                beer_name     anger   disgust      fear       joy   neutral  \
4     Castel Beer (Gabon)  0.009796  0.023428  0.002916  0.254752  0.699246   
5     Castel Beer (Gabon)  0.020531  0.110960  0.

In [98]:
# Emotions percentage in 10 % low review
low_score_reviews.groupby("beer_name")["max_feel"].value_counts(normalize=True) * 100

beer_name                      max_feel
ABC Extra Stout                disgust      56.321839
                               neutral      28.735632
                               joy           6.896552
                               anger         5.747126
                               sadness       1.149425
                                              ...    
Whitewater Snake Drive Bitter  disgust      50.000000
Whitewater Solstice            disgust      84.615385
                               joy           7.692308
                               neutral       7.692308
Whitewater St Donard           joy         100.000000
Name: proportion, Length: 795, dtype: float64

In [97]:
# Emotions percentage in all reviews
data.groupby("beer_name")["max_feel"].value_counts(normalize=True) * 100

beer_name                      max_feel
ABC Extra Stout                disgust      50.340136
                               neutral      27.891156
                               joy          11.564626
                               anger         5.442177
                               sadness       3.401361
                                              ...    
Whitewater Snake Drive Bitter  joy          50.000000
Whitewater Solstice            disgust      84.615385
                               neutral       7.692308
                               joy           7.692308
Whitewater St Donard           joy         100.000000
Name: proportion, Length: 1081, dtype: float64